# Use Case : churn rate

In [1]:
import pandas as pd
import os
import seaborn as sns
import numpy as np
import warnings
warnings.filterwarnings("ignore")

In [2]:
transactions = pd.read_csv(os.path.join("data", "client_history.csv")).drop(columns="Unnamed: 0")
transactions["date_order"] = pd.to_datetime(transactions["date_order"])

In [3]:
transactions.head()

,client_id,branch_id,date_order,order_invoice_delta,sales_net,quantity,past_lifetime_value,last_order,time_since_last,age,sales_net_growth_rate,churned
0,585799,5274,2017-09-22,3,654.616800,12,654.616800,0,NaN,0,NaN,0
1,585799,5274,2017-09-25,0,753.348900,53,1407.965700,0,3.0,3,0.15,0
2,585799,5274,2017-09-27,0,151.666600,16,1559.632300,0,2.0,5,-0.80,0
3,585799,5274,2017-09-28,1,436.930989,975,1996.563289,0,1.0,6,1.88,0
4,585799,5274,2017-09-29,3,134.716980,41,2131.280269,0,1.0,7,-0.69,0


## 1. Define churn

Feature ideas:

- past lifetime value ✅
- time since last purchase
- frequency acceleration --> do 
- future lifetime value
- is this the last purchase ✅
- churn --> lost client (to define)

In [59]:
client_transactions = transactions[transactions.client_id==585799]
client_transactions.sort_values(by="date_order", inplace=True)

In [60]:
client_transactions

,date_order,client_id,branch_id,sales_net,sales_net_growth_rate,past_lifetime_value,time_since_last,age,last_order
0,2017-09-22,585799,5274,654.616800,NaN,654.616800,NaN,0,0
1,2017-09-25,585799,5274,753.348900,0.2,1407.965700,3.0,3,0
2,2017-09-27,585799,5274,151.666600,-0.8,1559.632300,2.0,5,0
3,2017-09-28,585799,5274,436.930989,1.9,1996.563289,1.0,6,0
4,2017-09-29,585799,5274,134.716980,-0.7,2131.280269,1.0,7,0
...,...,...,...,...,...,...,...,...,...
181,2019-09-06,585799,5274,191.709600,-0.9,136205.640969,2.0,714,0
182,2019-09-08,585799,5274,303.572400,0.6,136509.213369,2.0,716,0
183,2019-09-12,585799,5274,1754.805568,4.8,138264.018938,4.0,720,0
184,2019-09-16,585799,5274,465.435360,-0.7,138729.454298,4.0,724,0


In [49]:
transactions.date_order.max()

Timestamp('2019-09-22 00:00:00')

In [72]:
transactions.churned.value_counts()

0    702859
1       885
Name: churned, dtype: int64

## 2. Real churn rate analysis

In [4]:
# After how many transactions did a ClientCo client churned
transactions.churned.value_counts()

0    702859
1       885
Name: churned, dtype: int64

## 3. Churn prediction model